In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

In [2]:

memory = MemorySaver()

class State(TypedDict):
    value: str

def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b", 
        update={
            "value": state["value"] + "a"
        }
    )

def node_b(state: State): 
    print("Node B")

    human_response = interrupt('would u like to go to c or d')

    print("Human Review Values: ", human_response)
    
    if(human_response == "C"): 
        return Command(
            goto="node_c", 
            update={
                "value": state["value"] + "b"
            }
        ) 
    elif(human_response == "D"): 
        return Command(
            goto="node_d", 
            update={
                "value": state["value"] + "b"
            }
        )
def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def node_d(state: State): 
    print("Node D")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a") 

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

initialState = {
    "value": ""
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result

Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='would u like to go to c or d', id='b1bfa2ec0a75dbbe50de3da33a772c11'),)}]

In [7]:
a=app.get_state_history(config=config)
for i in a:
    print(i)

StateSnapshot(values={'value': 'abc'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfa3-c5da-62aa-8003-36f750473dc8'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-19T12:44:24.492292+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfa3-c5d6-604c-8002-d194af6ca999'}}, tasks=(), interrupts=())
StateSnapshot(values={'value': 'ab'}, next=('node_c',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfa3-c5d6-604c-8002-d194af6ca999'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-19T12:44:24.490592+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07cfa2-05ac-6bd2-8001-05f228ac86d6'}}, tasks=(PregelTask(id='4df825d1-6773-83e4-b9fa-8b0c424504d2', name='node_c', path=('__pregel_pull', 'node_c'), error=None, interrupts=(), state=None, result={'val

In [4]:
app.get_state(config=config).next

('node_b',)

In [6]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
Human Review Values:  C
Node C


[{'node_b': {'value': 'ab'}}, {'node_c': {'value': 'abc'}}]